### TODO :
1. load data 
 - standardize images
 - split into train/val sets
2. load model
3. Loss function
 - cross entropy
4. optimization
 - SGD(momentum=0.99, learning_rate=0.01)
 - optimizer.zero_grad()
5. metrics(loss, accuracy)
 - each of train/val
6. save models and logs in middle of train
7. train on GPU

In [1]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.functional as F

import pathlib

import pandas as pd
from tqdm import tqdm
import numpy as np

# import import_ipynb
from steel_dataset import Steel_dataset
from model import U_net
from util import csv_file_load

from pre_processing import Pre_process_img

from collections import OrderedDict
import math

In [2]:
ROOT_PATH = pathlib.Path('severstal-steel-defect-detection')
IMG_FILE_PATH = ROOT_PATH / 'train_images'
TRAIN_FILE = ROOT_PATH / 'train.csv'
CK_PATH = pathlib.Path('checkpoints')

SPILIT_RATIO = 0.2
n_batch = 1

In [3]:
train_pd = csv_file_load(TRAIN_FILE)
val_idx = int(len(train_pd) * SPILIT_RATIO)

train_pd, val_pd = train_pd.iloc[:val_idx, :], train_pd.iloc[val_idx, :]

In [4]:
train_dataset = Steel_dataset(IMG_FILE_PATH, train_pd, out_size=(132, 1476))
val_dataset = Steel_dataset(IMG_FILE_PATH, val_pd, out_size=(132, 1476))

In [5]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=n_batch, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=n_batch, shuffle=True)

In [6]:
device = torch.device('cuda')

u_net = U_net(n_classes=5).to(device)

In [7]:
#out = u_net(train_dataset[0][0].unsqueeze(dim=0).to(device))

In [11]:
# from IPython.core.debugger import set_trace
# set_trace()
(train_dataset[0][0].numel() + train_dataset[0][1] .numel()) * 3 / 1024 / 1024

5.05975341796875

In [12]:
train_dataset[0][0].size() == train_dataset[0][1].size()

False

In [9]:
def save_model(model, optim, save_path, epoch, loss):
    torch.save({
        'model_state_dict': model.state_dict(),
        'epoch': epoch,
        'loss' : loss,
        'optim_state_dict': optim.state_dict()
    }, save_path)

In [14]:
class_weight = np.array((train_pd.groupby('ClassId').count().apply(lambda x: x/x.sum() ).iloc[:,0]*100).astype(int))

print(class_weight)

[13  4 71 10]


In [13]:
criterion = nn.CrossEntropyLoss()
optim = torch.optim.SGD(u_net.parameters(), momentum=0.99, lr=0.001)

In [ ]:
n_epochs = 10

for epoch in tqdm(range(n_epochs)):
    for i, (x_, mask) in enumerate(train_dataloader):
        x_ = x_.cuda()
        mask = mask.cuda().long()
        from IPython.core.debugger import set_trace
        set_trace()
        optim.zero_grad()
      
        out = u_net(x_)
        loss_train = criterion(out, mask)
        loss_train.backward()
        optim.step()
    
    
    with torch.no_grad():
        for j, (val_x, val_mask) in enumerate(val_dataloader):
            out_val = u_net(val_x)
            loss_val = criterion(out_val, val_mask)
            
    print(f'epoch:{epoch} batch:{i}th \n -- train_loss:{loss_train} \
                        train_accuracy:{(torch.argmax(out, dim=1).squeeze() == mask).sum() / float(out.numel()) * 100}')
    
    print(f'epoch:{epoch} batch:{i}th \n -- val_loss:{loss_val} \
                          train_accuracy:{(torch.argmax(out_val, dim=1).squeeze() == val_mask).sum() / float(out_val.numel()) * 100}')        
    
    save_model(u_net, optim, CK_PATH / f'u_net_{epoch}e_{int(loss_val)}l.pt')

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

> <ipython-input-15-63dd1895c2a0>(9)<module>()
      7         from IPython.core.debugger import set_trace
      8         set_trace()
----> 9         optim.zero_grad()
     10 
     11         out = u_net(x_)



ipdb>  n


> <ipython-input-15-63dd1895c2a0>(11)<module>()
      9         optim.zero_grad()
     10 
---> 11         out = u_net(x_)
     12         loss_train = criterion(out, mask)
     13         loss_train.backward()



ipdb>  n


...

> <ipython-input-15-63dd1895c2a0>(13)<module>()
     11         out = u_net(x_)
     12         loss_train = criterion(out, mask)
---> 13         loss_train.backward()
     14         optim.step()
     15 



ipdb>  n


RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 2.00 GiB total capacity; 1.10 GiB already allocated; 114.50 MiB free; 1.18 GiB reserved in total by PyTorch)
> <ipython-input-15-63dd1895c2a0>(13)<module>()
     11         out = u_net(x_)
     12         loss_train = criterion(out, mask)
---> 13         loss_train.backward()
     14         optim.step()
     15 



ipdb>  out.element_size()


4


In [30]:
# print(u_net.state_dict())
total_ = 0
for k, v in u_net.state_dict().items():
    print(f'name:{k} size:{v.size()} dtype:{v.dtype}')
    total_ += v.numel()
print(f'Model size : {total_*4} byte -> {total_*4/1024**2} MiB')


name:cont_layer1.c_conv11.weight size:torch.Size([64, 3, 3, 3]) dtype:torch.float32
name:cont_layer1.c_conv11.bias size:torch.Size([64]) dtype:torch.float32
name:cont_layer1.c_conv12.weight size:torch.Size([64, 64, 3, 3]) dtype:torch.float32
name:cont_layer1.c_conv12.bias size:torch.Size([64]) dtype:torch.float32
name:cont_layer2.c_conv21.weight size:torch.Size([128, 64, 3, 3]) dtype:torch.float32
name:cont_layer2.c_conv21.bias size:torch.Size([128]) dtype:torch.float32
name:cont_layer2.c_conv22.weight size:torch.Size([128, 128, 3, 3]) dtype:torch.float32
name:cont_layer2.c_conv22.bias size:torch.Size([128]) dtype:torch.float32
name:cont_layer3.c_conv31.weight size:torch.Size([256, 128, 3, 3]) dtype:torch.float32
name:cont_layer3.c_conv31.bias size:torch.Size([256]) dtype:torch.float32
name:cont_layer3.c_conv32.weight size:torch.Size([256, 256, 3, 3]) dtype:torch.float32
name:cont_layer3.c_conv32.bias size:torch.Size([256]) dtype:torch.float32
name:cont_layer4.c_conv41.weight size:torc